In [ ]:
# get tflite file (pretrained) https://tfhub.dev/tensorflow/lite-model/efficientdet/lite0/detection/default/1

In [2]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 642 kB 7.4 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
     |████████████████████████████████| 10.9 MB 53.0 MB/s 
     |████████████████████████████████| 128 kB 70.9 MB/s 
     |████████████████████████████████| 1.1 MB 59.5 MB/s 
     |████████████████████████████████| 238 kB 74.1 MB/s 
     |████████████████████████████████| 840 kB 51.9 MB/s 
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |████████████████████████████████| 55.3 MB 190 bytes/s 
     |████████████████████████████████| 78 kB 7.7 MB/s 
     |████████████████████████████████| 1.2 MB 64.6 MB/s 
     |████████████████████████████████| 3.4 MB 50.3 MB/s 
     |████████████████████████████████| 25.3 MB 88.8 MB/s 
     |████████████████████████████████| 352 kB 70.2 MB/s 
     |████████████████████████████████| 99 kB 10.9 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 40 kB 5.7 MB/s 
     |█████████████

In [3]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [4]:
# download dataset
!wget https://storage.googleapis.com/download.tensorflow.org/data/android_figurine.zip
!unzip -q android_figurine.zip

--2022-08-06 02:10:34--  https://storage.googleapis.com/download.tensorflow.org/data/android_figurine.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14333895 (14M) [application/zip]
Saving to: ‘android_figurine.zip’

android_figurine.zi 100%[===================>]  13.67M  67.9MB/s    in 0.2s    

2022-08-06 02:10:35 (67.9 MB/s) - ‘android_figurine.zip’ saved [14333895/14333895]



### Load dataset

In [5]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'android_figurine/train',
    'android_figurine/train',
    ['android', 'pig_android']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'android_figurine/validate',
    'android_figurine/validate',
    ['android', 'pig_android']
)

### Select model

In [6]:
spec = model_spec.get('efficientdet_lite0')

### Train model

In [7]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
15/15 [==============================] - 62s 878ms/step - det_loss: 1.7074 - cls_loss: 1.1220 - box_loss: 0.0117 - reg_l2_loss: 0.0630 - loss: 1.7704 - learning_rate: 0.0065 - gradient_norm: 2.0717 - val_det_loss: 1.5630 - val_cls_loss: 1.0846 - val_box_loss: 0.0096 - val_reg_l2_loss: 0.0630 - val_loss: 1.6260
Epoch 2/20
15/15 [==============================] - 10s 705ms/step - det_loss: 1.4782 - cls_loss: 1.0217 - box_loss: 0.0091 - reg_l2_loss: 0.0630 - loss: 1.5412 - learning_rate: 0.0049 - gradient_norm: 2.5030 - val_det_loss: 1.2635 - val_cls_loss: 0.8849 - val_box_loss: 0.0076 - val_reg_l2_loss: 0.0630 - val_loss: 1.3265
Epoch 3/20
15/15 [==============================] - 9s 643ms/step - det_loss: 1.1864 - cls_loss: 0.8170 - box_loss: 0.0074 - reg_l2_loss: 0.0630 - loss: 1.2494 - learning_rate: 0.0048 - gradient_norm: 2.8922 - val_det_loss: 0.9984 - val_cls_loss: 0.6831 - val_box_loss: 0.0063 - val_reg_l2_loss: 0.0630 - val_loss: 1.0614
Epoch 4/20
15/15 [==============

### Evaluation

In [8]:
model.evaluate(val_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.81722534,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/android': 0.78518456,
 'AP_/pig_android': 0.84926605,
 'APl': 0.81722534,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.8451389,
 'ARm': -1.0,
 'ARmax1': 0.8277778,
 'ARmax10': 0.8451389,
 'ARmax100': 0.8451389,
 'ARs': -1.0}

### Export

In [9]:
model.export(export_dir='.', tflite_filename='android-figurine.tflite')

### Evaluate TFLite

In [10]:
model.evaluate_tflite('android-figurine.tflite', val_data)

10/10 [==============================] - 37s 3s/step



{'AP': 0.82412386,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/android': 0.8121169,
 'AP_/pig_android': 0.83613074,
 'APl': 0.82412386,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.8347222,
 'ARm': -1.0,
 'ARmax1': 0.8347222,
 'ARmax10': 0.8347222,
 'ARmax100': 0.8347222,
 'ARs': -1.0}

In [12]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('android-figurine.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>